In [13]:
import numpy as np
import pandas as pd 
from google import genai
from google.genai import types as gt
import os
from pydantic import create_model
import json


In [14]:
#Credentials set up

#Local
from dotenv import load_dotenv
load_dotenv('../.env',override=True)

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

#Kaggle
# from kaggle_secrets import UserSecretsClient
# GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

client = genai.Client(api_key = GOOGLE_API_KEY)

In [15]:
main_df = pd.read_csv('../data/main_df.csv')

deciles_features_list = ['danceability', 'energy', 'speechiness','acousticness', 'instrumentalness', 'liveness', 'valence','views']
direct_use_features = ['loudness','tempo','duration_ms']


fields = {}

for feature in deciles_features_list:
    fields[feature+"_min_decile"] = (int,0)
    fields[feature+"_max_decile"] = (int,10)
    fields[feature+"_decile_weight"] = (int,0)


for feature in direct_use_features:
    fields[feature+"_min"] = (int,-100)
    fields[feature+"_max"] = (int,99999999)
    fields[feature+"_decile_weight"] = (int,0)

minmax_only_features = ['album_release_year','track_is_explicit']

fields['album_release_year_min'] = (int,1900)
fields['album_release_year_max'] = (int,2025)

fields['track_is_explicit_min'] = (int,0)
fields['track_is_explicit_max'] = (int,1)

fields['spotify_artist_genres_include_any'] = (str,'')
fields['spotify_artist_genres_exclude_any'] = (str,'')
fields['spotify_artist_genres_boosted'] = (str,'')
fields['debug_tag'] = (str,'')
fields['reflection'] = (str,'')
fields['user_message'] = (str,'')

FiltersModel = create_model("Filters", **fields)

# print(FiltersModel.model_json_schema())

In [16]:
## Explode genres to understand which to whitelist for the llm

g = (main_df
       .assign(genre_list=lambda d: d["spotify_artist_genres"]
                                   .fillna("")
                                   .str.split(","))          # list
       .explode("genre_list")                                # <-- explode the list col
       .assign(genre=lambda d: d["genre_list"]
                               .str.strip()
                               .str.lower())
       .loc[lambda d: d["genre"] != ""])                     # drop empties

# 2. Aggregate by the single-genre column
stats = (g.groupby("genre")
           .agg(tracks=("spotify_track_id", "nunique"),
                views_sum=("views", "sum"),
                views_mean=("views", "mean"),
                streams_sum=("stream", "sum"),
                streams_mean=("stream", "mean"))
           .reset_index())

# 3. Score & pick allow list
stats["freq_z"]  = (stats["tracks"]    - stats["tracks"].mean())    / stats["tracks"].std()
stats["views_z"] = (stats["views_sum"] - stats["views_sum"].mean()) / stats["views_sum"].std()
stats["score"]   = 0.6*stats["freq_z"] + 0.4*stats["views_z"]

allow_list = (stats.sort_values("score", ascending=False)
                    .head(100)["genre"]
                    .tolist())

stats.to_csv('../data/genre_stats.csv', index=False)

In [ ]:

SYSTEM_INSTRUCTION = """
Your task is to convert a user query to a set of filters we can use to query a music database. 
The database has about 18,000 tracks. Aim to design filters that will return a result set of about 50-150 tracks, as long as they are relevant to the user's query.

Below is a list of features we can use to filter as well as to create a relevance score. Note many of these are correlated. 

For the following features, you can create min/max filters and also a weight for relevance scoring.
danceability_decile: Danceability describes how suitable a track is for dancing. The higher the number the higher the dancability. Converted to deciles (1-10).
speechiness_decile: Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the higher the attribute value. Converted to deciles (1-10).
energy_decile: represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Converted to deciles (1-10).
acousticness_decile: A confidence measure of whether the track is acoustic. The higher the number the higher the confidence. Converted to deciles (1-10).
instrumentalness_decile: Predicts whether a track contains no vocals. "Ooh" and "aah" sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly "vocal". The higher the instrumentalness value is, the greater likelihood the track contains no vocal content. Deciles are not balanced and start at 5 for definitely vocal tracks, max is 10.
liveness_decile: Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. Converted to deciles (1-10).
valence_decile: A measure describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). Converted to deciles (1-10).
views_decile: the number of YouTube views the music has. Converted to deciles (1-10).
tempo: The overall estimated tempo of a track in beats per minute (BPM). Consider BPMs under 60 ultra slow and hard to detect a beat. 60-90 are slow with a beat detectable. 90-120 are mid tempo/pop. 120-140 are fast and upbeat. 140 and above is very fast. Min/max filters are not applied against deciles but againt the raw value. The score weight will be multiplied by the decile.
loudness: The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Note that overall loudness is normalized across tracks on Spotify, so only use this in extreme scenarios. Min/max filters are not applied against deciles but againt the raw value. The score weight will be multiplied by the decile.
duration_ms: the duration of the track in milliseconds. Always ensure to provide at least 3 seconds of a range if you use this filter. Min/max filters are not applied against deciles but againt the raw value. The score weight will be multiplied by the decile.

For the following features, you can only create mix/max filters:
album_release_year: the year the track was released in, represented as an integer. This is the year the recording was made, do not rely on this for classical music or any music from before 1950. 
track_is_explicit: this is 0 for tracks without explicit language and 1 for tracks marked as explicit language. 

For 60% of tracks, we were also able to pull the artist's genres into a comma separated list field. You can use these to create filters and scoring against that field. Note that any filtering will only apply to the 60% of tracks that have genres. Only use this field if the user gives a clear indication of style or genre required. 
Values you can choose and are common are: pop, rock, r&b, hip hop, rap, edm, house, reggaeton, latin, country, k-pop, bollywood, metal, disco, orchestra, classical.
Values you can choose that are more rare: ambient, soundtrack, lo-fi, drum and bass, christmas, children, anime, emo, piano. Use these sparingly. 
There are a lot more genres but they are even more rare. Only use a genre outside the list above if the user explicitly asks for it. 
spotify_artist_genres_include_any: Provide a comma separated string to filter in tracks that have at least one of your genres included. If you include a substring like pop it will also capture genres like k-pop. Be as general as possible. If you use more niche genres, make sure to widen other filters. 
spotify_artist_genres_exclude_any: Provide a comma separated string to filter out tracks that have any of your genres included. If you exclude a substring like pop it will also exclude genres like k-pop. Be as specific as possible. Use sparingly. 
spotify_artist_genres_boosted: Provide a comma separated string to give a 50 point relevance score boost to tracks that have any included genre. Artists with multiple included genres will get multiple boosts. 

Please return JSON following the schema provided.
Attributes with a min or max in the name will create a filter for the user. 
Attributes with weight or boost in the name will contribute to a relevance score to sort results. 
The weight will be multiplied by the decile of that feature to create the score.
Please make it a number between -100 and 100.
Attributes with include all, include any, or exclude will create a filter for the user.

Please return values for all attributes in the schema.
If you do not want to filter on a particular attribute, use the default values for min and max to ensure all values are included.
If you do not want an attribute to contribute the the relevance score, return 0 for its weight.
Include the tag SYS_TAG_8424 as a string under the field debug_tag.

Additional fields in the schema for you to provide:
reflection: summarize key decisions you made on how you set or changed filters. Which are most important and why? What additional filters could make this better? What would you try next to refine the results?
user_message: provide a brief 1-2 sentence explanation of the filters you provided to the user and why you chose them. In a 3rd sentence, ask the user a specific question that you think will help you refine the results further.
"""


def llm_query(latest_prompt,history=None):
    print('sending to LLM')
    cfg = gt.GenerateContentConfig(
        system_instruction=SYSTEM_INSTRUCTION,
        response_mime_type="application/json",
        response_schema=FiltersModel,
    )
    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=(history or []) + [latest_prompt],
        config=cfg
    )
    print('LLM response received',json.loads(response.text))
    return json.loads(response.text)

In [26]:
def _split_terms(s):
    return [t.strip() for t in s.split(",")] if s else []

def llm_to_filters(response_json):
    filters_object = response_json
    combined_filter = pd.Series(True, index=main_df.index)
    for feature in deciles_features_list:
        if filters_object[feature+'_min_decile'] and filters_object[feature+'_max_decile']:
            combined_filter = combined_filter & \
                (main_df[feature+'_decile'] >= filters_object[feature+'_min_decile']) & \
                (main_df[feature+'_decile'] <= filters_object[feature+'_max_decile'])
    
    for feature in direct_use_features + minmax_only_features:
        if filters_object[feature+'_min'] and filters_object[feature+'_max']:
            combined_filter = combined_filter & \
                (main_df[feature] >= filters_object[feature+'_min']) & \
                (main_df[feature] <= filters_object[feature+'_max'])


    if filters_object['spotify_artist_genres_include_any'] and len(filters_object['spotify_artist_genres_include_any']) > 0:
        included_terms = _split_terms(filters_object['spotify_artist_genres_include_any'])
        if included_terms:
            combined_filter = combined_filter & \
                main_df['spotify_artist_genres'].fillna("").apply(lambda g: any(term in g for term in included_terms))
    
    if filters_object['spotify_artist_genres_exclude_any'] and len(filters_object['spotify_artist_genres_exclude_any']) > 0:
        excluded_terms = _split_terms(filters_object['spotify_artist_genres_exclude_any'])
        if excluded_terms:
            combined_filter = combined_filter & \
                main_df['spotify_artist_genres'].fillna("").apply(lambda g: not any(term in g for term in excluded_terms))
    
    
    print('Filter created')
    return combined_filter

In [27]:
GENRE_BOOST_POINTS = 50

def filters_to_results_df(combined_filter,filters_object):
    filtered_results = main_df[combined_filter].copy()

    relevance_weights = []
    cols = deciles_features_list + direct_use_features
    for feature in cols:
        if filters_object[feature+'_decile_weight']:
            relevance_weights.append(filters_object[feature+'_decile_weight'])
        else:
            relevance_weights.append(0)

    filtered_results['relevance_score'] = filtered_results[deciles_features_list + direct_use_features].mul(relevance_weights).sum(axis=1)

    boost_terms = _split_terms(filters_object.get('spotify_artist_genres_boosted',''))

    if boost_terms:
        filtered_results["genre_boost_hits"] = filtered_results["spotify_artist_genres"].fillna("").apply(
            lambda g: sum(term in g for term in boost_terms)
        )

        filtered_results["relevance_score"] += GENRE_BOOST_POINTS * filtered_results["genre_boost_hits"]
    
    return filtered_results

In [28]:
TOP_K = 5
EXAMPLE_COLS = [
    "spotify_track_id", "track", "artist","spotify_artist_genres",
    'danceability_decile', 'energy_decile', 'speechiness_decile','acousticness_decile', 'instrumentalness_decile', 'liveness_decile', 'valence_decile','views_decile',
    'loudness', "tempo",
    "album_release_year", "duration_ms", "track_is_explicit",
    "relevance_score",
    "url_youtube"
]

def truncate(s, n=120):
    return (s[:n] + "…") if isinstance(s, str) and len(s) > n else s

def make_summary(df, top_k=TOP_K):
    top = df.sort_values("relevance_score", ascending=False).head(top_k).copy()
    if "description" in df.columns:
        top["description_short"] = top["description"].apply(truncate)

    summary = {}
    if int(len(df)) == 0:
        summary = {
            "result_count": int(len(df))
        }
    else:
        top_examples = top[EXAMPLE_COLS + (["description_short"] if "description_short" in top else [])] \
            .to_dict(orient="records")
        summary = {
            "result_count": int(len(df)),
            "top_examples": top_examples,
            "score_stats": {
                "min": float(df["relevance_score"].min()),
                "median": float(df["relevance_score"].median()),
                "max": float(df["relevance_score"].max())
            },
            "year_range": {
                "min": int(df["album_release_year"].min()),
                "max": int(df["album_release_year"].max())
            },
            "top_genres_found": (
                df["spotify_artist_genres"].str.split(",")
                  .explode()
                  .str.strip()
                  .value_counts()
                  .head(5)
                  .index.tolist()
            )
        }
    return summary


In [21]:
from IPython.display import YouTubeVideo, display, HTML
def extract_yt_id_from_url(url):
    return url.strip('https://www.youtube.com/watch?v=')

def filtered_results_to_vids(filtered_results,n,llm_return):
    top_n_results = filtered_results.sort_values(by='relevance_score', ascending=False)[0:n]
    # youtube_urls = top_n_results.url_youtube
    # youtube_ids =list(map(extract_yt_id_from_url,youtube_urls))
    # print("https://www.youtube.com/watch_videos?video_ids=" + ",".join(youtube_ids))
    # for video_id in youtube_ids:
    #     display(YouTubeVideo(video_id, width=300, height=150))  
    
    html_content = f"""<div>
    <h1>{len(filtered_results)} Results</h1>
    <p>User message: {llm_return.get('user_message')}</p>
    <p>Reflection: {llm_return.get('reflection')}</p>
    """;
    i = 0
    for index, row in top_n_results.iterrows():
        i = i+1
        html_content += f"""
        <div style="border-bottom: 1px solid #ddd;padding-bottom:10px;margin-bottom:10px">
            <h2><a href='{row.url_youtube}' target='_blank'>{i}. {row['track']} by {row['artist']}</a></h2>
            <h3><a href='https://open.spotify.com/track/{row.spotify_track_id}' target='_blank'>Spotify</a> | Released {row['album_release_year']} | Genres: {row['spotify_artist_genres']} | Explicit: {row['track_is_explicit']}</h3>
            <h4>
                Scores
            </h4>
            <ul>"""

        for feature in deciles_features_list:
            html_content += f"<li>{feature}: {row[feature+'_decile']}/10</li>"

        for feature in direct_use_features:
            html_content += f"<li>{feature}: {row[feature]}/10</li>"
            
        html_content += """
                </ul>
                <div style="text-align: left; margin-top: 15px;">
                    <iframe width="300" height="150" src="https://www.youtube.com/embed/{extract_yt_id_from_url(row['url_youtube'])}" allowfullscreen></iframe>
                </div>
            </div>
        """
    html_content += "</div>"
    return display(HTML(html_content))

In [22]:
def initial_prompt(user_query: str) -> str:
    return f"User query: {user_query}\nReturn ONLY JSON per schema."

TARGET_MIN, TARGET_MAX = 50, 150  # tweak

def refine_prompt(user_query: str, prev_json: dict, summary_json: dict, latest_user_feedback: str = None) -> str:
    text = f"Refine your previous JSON to better match the user intent.\n"
    text += f"Aim to have between {TARGET_MIN} and {TARGET_MAX} results. Inspect the top 10 results to ensure they are relevant and also of high quality.\n"
    text += f"Tighten or relax your criteria as needed to reach this target, unless quality and relevance are clearly degrading.\n\n"
    text += f"Original user query: {user_query}\n"
    if latest_user_feedback:
        text += f"Latest user feedback: {latest_user_feedback}\n"
    text += f"Previous JSON: {json.dumps(prev_json)}\n"
    text += f"Summary: {json.dumps(summary_json)}\n\n"
    text += "Return ONLY JSON per schema."
    return text


def run_auto_refine(user_query: str, max_iters=3):
    # ---- Turn 0: cold start ----
    filters_json = llm_query(initial_prompt(user_query))

    for i in range(max_iters - 1):
        mask = llm_to_filters(filters_json)
        results = filters_to_results_df(mask, filters_json)

        count = len(results)
        print("result count", count)
        if TARGET_MIN <= count <= TARGET_MAX:
            break

        summary = make_summary(results)
        filters_json = llm_query(
            refine_prompt(user_query, filters_json, summary),
        )

    # final results
    mask = llm_to_filters(filters_json)
    results = filters_to_results_df(mask, filters_json)
    count = len(results)
    print("result count", count)
    return filters_json, results

In [30]:
from IPython.display import YouTubeVideo, display, HTML

history = []
query = input("What kind of music can I help you find?")
# query = 'Music for a period drama set in elizabethan england'
# query_to_parameters = llm_query(query)
# parameters_to_filters = llm_to_filters(query_to_parameters)
# results = filters_to_results_df(parameters_to_filters,query_to_parameters)


filters_json, results = run_auto_refine(query)
history.append([query,filters_json.get('user_message')])

filtered_results_to_vids(results,10,filters_json)

sending to LLM
LLM response received {'danceability_min_decile': 1, 'danceability_max_decile': 10, 'danceability_decile_weight': 0, 'energy_min_decile': 1, 'energy_max_decile': 10, 'energy_decile_weight': 0, 'speechiness_min_decile': 1, 'speechiness_max_decile': 2, 'speechiness_decile_weight': -70, 'acousticness_min_decile': 1, 'acousticness_max_decile': 10, 'acousticness_decile_weight': 0, 'instrumentalness_min_decile': 9, 'instrumentalness_max_decile': 10, 'instrumentalness_decile_weight': 100, 'liveness_min_decile': 1, 'liveness_max_decile': 10, 'liveness_decile_weight': 0, 'valence_min_decile': 1, 'valence_max_decile': 10, 'valence_decile_weight': 0, 'views_min_decile': 1, 'views_max_decile': 10, 'views_decile_weight': 0, 'loudness_min': -60, 'loudness_max': 0, 'loudness_decile_weight': 0, 'tempo_min': 0, 'tempo_max': 250, 'tempo_decile_weight': 0, 'duration_ms_min': 0, 'duration_ms_max': 6000000, 'duration_ms_decile_weight': 0, 'album_release_year_min': 1900, 'album_release_year_m

In [24]:
next_query = input("How does that look? Any changes?")
filters_json = llm_query(
    refine_prompt(query, filters_json, make_summary(results),next_query),
)

# final results
mask = llm_to_filters(filters_json)
results = filters_to_results_df(mask, filters_json)
filtered_results_to_vids(results,10,filters_json)

sending to LLM
LLM response received {'danceability_min_decile': 1, 'danceability_max_decile': 10, 'danceability_decile_weight': 0, 'energy_min_decile': 1, 'energy_max_decile': 10, 'energy_decile_weight': 0, 'speechiness_min_decile': 1, 'speechiness_max_decile': 2, 'speechiness_decile_weight': -50, 'acousticness_min_decile': 1, 'acousticness_max_decile': 10, 'acousticness_decile_weight': 0, 'instrumentalness_min_decile': 9, 'instrumentalness_max_decile': 10, 'instrumentalness_decile_weight': 80, 'liveness_min_decile': 1, 'liveness_max_decile': 10, 'liveness_decile_weight': 0, 'valence_min_decile': 1, 'valence_max_decile': 10, 'valence_decile_weight': 0, 'views_min_decile': 5, 'views_max_decile': 10, 'views_decile_weight': 30, 'loudness_min': -60, 'loudness_max': 0, 'loudness_decile_weight': 0, 'tempo_min': 0, 'tempo_max': 250, 'tempo_decile_weight': 0, 'duration_ms_min': 0, 'duration_ms_max': 9999999, 'duration_ms_decile_weight': 0, 'album_release_year_min': 1950, 'album_release_year_m